In [2]:
import pandas as pd
import random #Randomize the time
import requests
from time import sleep #Time delay for webscraping

from bs4 import BeautifulSoup as soup #Webscaping
import sqlite3 
from api_key import key #Get the API key from themoviedb.com and put in api_key file

#Import all the function needed to answer the actor/actress question
#Contain: castRankList(ID), castRank(Name), cast(ID)
from castFunctions import functions

#Retrive the key from api_key
api_key = key

1) SQL Database 

In [40]:
#Scrape 10 different pages to get the list of 1000 top actors
nameList=pd.DataFrame({"Rank":[],"Name":[] })
for pageindex in [1,2,3,4,5,6,7,8,9,10]:
    url="https://www.imdb.com/list/ls058011111/?sort=list_order,asc&mode=detail&page="+str(pageindex)+"&ref_=nmls_vm_dtl"
    response = requests.get(url)
    html_soup=soup(response.text, "html.parser")
    page=html_soup.find_all(class_="lister-item-content")
    for index, element in enumerate(page):
        nameList.loc[len(nameList)]=[index+1+(pageindex-1)*100,element.find_all('a')[0].text[:-1]]
    #This function randomize the delay time to avoid overload a website
    sleep(random.random())

In [41]:
#Create a rank List database
!rm 'actorRank.db' #Avoid duplicating the file
conn = sqlite3.connect('Data/actorRank.db')
cur=conn.cursor()

#Create table "rank" in the "actorRank" database. This have 2 columns: Rank, Name
cur.execute("""CREATE TABLE rank(Rank Integer, Name TEXT);""")

#Insert values from the "nameList" created above to the table
for i in range(nameList.shape[0]):
    name=nameList['Name'][i].replace("'"," ")
    rank=nameList['Rank'][i]
    #Note that there is a '' around {}
    cur.execute("""INSERT INTO rank (Rank,Name) VALUES ({},'{}');""".format(rank,name))

conn.commit() #Save the data base

2) Movies CVS

In [3]:
#Given a year, return data frame of that year

def moviesData(year):
    #URL manipulation
    urlDiscover='https://api.themoviedb.org/3/discover/movie?api_key='
    searchkey='&primary_release_year='+str(year)+'&sort_by=revenue.desc'
    response = requests.get(urlDiscover + api_key + searchkey)
    revenueResult = response.json() 
    revenueResultLimit= revenueResult['results']
    
    #Create df columns
    df = pd.DataFrame(columns=['Release_Date','ID','Title','Budget','Revenue','Profit','Runtime','Rating','Genres','Sub-genres'])
    #Update the columns
    for film in revenueResultLimit:
        filmDetails = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key +'&language=en-US')
        filmDetails = filmDetails.json()
        profit=filmDetails['revenue']-filmDetails['budget']
        df.loc[len(df)]=[film['release_date'],film['id'],film['title'],filmDetails['budget'],filmDetails['revenue'], profit,filmDetails['runtime'],filmDetails['vote_average'],filmDetails['genres'][0]['name'],subgenres(filmDetails)]
    return df

In [4]:
#This is a supplement function for the above function
#Give the json file "filmDetails", return the list of all genres
def genres(filmDetails):
    genresList=[]
    for i in range(len(filmDetails['genres'])):
        genresList.append(filmDetails['genres'][i]['name'])
    return genresList

In [5]:
#Given movies detail, return the second genres in the genres list
def subgenres(filmDetails):
    if len(filmDetails['genres'])>1:
        return filmDetails['genres'][1]['name']
    else:
        return ""

In [6]:
#Given ID, return US certification
def certification(id):
    url="https://api.themoviedb.org/3/movie/"+str(id)+"/release_dates?api_key=25e030646533a9beb625382caba1b356&language=en-US"
    response=requests.get(url)
    filmDetails=response.json()
    
    #Loop through the json and find MPAA
    for i in filmDetails['results']:
        if i["iso_3166_1"]=="US":
            for j in i["release_dates"]:
                if len(j["certification"])>0:
                    return j["certification"]
    return 0
    

In [7]:
#Create a list of Data Frame from 2009 to 2019
yearList=[]
for i in range(11):
    year=2009+i
    yearList.append(moviesData(year))
    print(year)
    sleep(8)

2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


In [10]:
#Create a df of all 10 years.
df=pd.concat(yearList,ignore_index=True)

In [11]:
#Get MPAA list using certification function. This take 2 min
MPAAList=[]
count=1
for i in df["ID"]:
    MPAAList.append(certification(i))
    count+=1
    if count%40==0:
        sleep(9) 

In [12]:
#Add MPAA column to data list
df["MPAA"]=MPAAList

In [14]:
#Add numTop1k column. This would take 1 min
f=functions()
numTop1k=[]
for id in df["ID"]:
    numTop1k.append(len(f.castRankList(id)))

In [15]:
#Add number of A list actors to data list
df["numTop1k"]=numTop1k

In [17]:
#Clean data
df=df[df["Title"]!="In The Mood For Surrealism or 8021"].reset_index(drop=True)
df=df[df["Title"]!="Medium"].reset_index(drop=True)

In [18]:
#Save that df in a csv file
!rm Data/moviesData.csv
df.to_csv('Data/moviesData.csv')

rm: moviesData.csv: No such file or directory
